# Exercice 2 : Régression et Algorithme de Longstaff-Schwartz

Nous importons les librairies dont nous avons besoin

In [29]:
import numpy as np
import time 
from scipy.stats import norm

Nous réutilisons la fonction de simulations Monte-Carlo de spots du sous-jacent 

In [30]:
def simulate_bs_process(S0=50., r=0.05, sigma=0.15, T=1., h=1./360):
    nb_sim = int(T / h) - 1
    
    if nb_sim <= 0:
        return [S0]
    
    gauss_mean = (r - 0.5 * sigma**2) * h
    gauss_stddev = sigma * np.sqrt(h)
    
    gauss_rvs = np.random.normal(
        loc=gauss_mean,
        scale=gauss_stddev,
        size=nb_sim)
    
    exp_gauss = np.exp(gauss_rvs)
    exp_gauss = np.insert(exp_gauss, 0, S0)

    return np.cumprod(exp_gauss)   

Nous définissons une matrice contenant l'ensemble des simulations de spot

In [31]:
nb_paths = 10000

time_vec = np.linspace(0., 1., 360)
sim_paths = np.array([simulate_bs_process() for _ in range(nb_paths)])

## Algorithme de Longstaff-Schwartz

Nous considérons le pricing d'un call européen.

Nous venons de simuler 10 000 trajectoires du sous-jacent du call, chaque trajectoire contenant 360 intervalles de temps. Ces spots de sous-jacent sont stockés dans une matrice nommée "spot". Nous créons une matrice "price", destinée à stocker les prix de l'option à chaque date de spot simulé.

Nous remplissons dans un premier temps la dernière colonne de "price", car à la date terminale le prix de l'option se calcule à partir du niveau du sous-jacent: p=(ST-K)+

Pour remplir les autres colonnes (les autres dates) de la matrice des prix de l'option, nous utilisons une régression aux moindres carrés polynomiale: pi+1 actualisé=c + alpha * s + beta * s².
Nous en déduisons le prix de l'option pour la trajectoire n: pi= c + alpha * s + beta * s².

Une fois cet algorithme arrivé à la date t0, pour déduire le prix de l'option à la date initiale, nous effectuons une moyenne arithmétique des prix de l'option à cette date: p0=1/N * somme(p0n).


In [32]:
n = 360
spot = sim_paths
price= np.zeros([sim_paths.shape[0],n])
maturity = 1

dt = 1./360
nbSim = 10000
S0 = 50
r = 0.05
sigma = 0.15
K = 49


t = np.linspace(0,maturity,n)
S = np.zeros([n]) 
S[0] = S0 


price = np.where(spot>K, spot-K, 0)
for m in range(1,n): 

        x_reg = spot[:,n-m-1] 
        y_reg = np.exp(-r*dt)*price[:,n-m]
        coeff_reg = np.polyfit(x_reg, y_reg, 2)

        price[:,n-m-1]=(coeff_reg[2]+coeff_reg[1]*spot[:,n-m-1]+coeff_reg[0]*spot[:,n-m-1]**2)


print(np.mean(price[:,0]))

4.816174615468328


C:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:24: RankWarning: Polyfit may be poorly conditioned


Nous vérifions la cohérence notre prix calculé grâce à Monte Carlo et Longstaff Schwartz en le comparant avec la formule fermée de Black & Scholes pour un call européen:

In [33]:
def bs_call_price(S=100., r=0.02, sigma=0.2, t=0., T=1., K=100.):
    ttm = T - t
    
    if ttm < 0:
        return 0.
    elif ttm == 0.:
        return np.maximum(S - K, 0.)

    vol = sigma * np.sqrt(ttm)

    d_minus = np.log(S / K) + (r - 0.5 * sigma**2) * ttm
    d_minus /= vol

    d_plus = d_minus + vol

    res = S * norm.cdf(d_plus)
    res -= K * np.exp(-r * ttm) * norm.cdf(d_minus)

    return res

In [34]:
bs_call_price(S0, r, sigma, 0, T, K)

4.892712596170217